# K-Nearest Neighbor (KNN)
> 회귀와 분류 모두 사용되는 지도학습 알고리즘으로  
> 다른 알고리즘에 비해 쉽지만 연산 속도가 느리다는 단점이 있다.

KNN 알고리즘을 설명할 때 유유상종이라는 말을 많이 인용한다.  
유유상종의 뜻 처럼 비슷한 특성 및 속성을 가진 데이터는 가깝게 모여서 존재하기 때문에 가장 가까운 K개의 속성에 따라서 분류하는 알고리즘이다.  
n_neighbors의 값이 작을수록 복잡한 모델, 클수록 단순한 모델이며 n_neighbors의 값을 절절하게 주어야 과대적합과 과소적합이 발생하지 않는다.

## n_neighbors 값의 중요성

- n_neighnors는 탐색하는 이웃의 개수로 n_neighnors에 따라 데이터를 다르게 예측한다.
- 일반적으로 n_neighnors를 1이 아닌 홀수로 설정한다.
    - 1로 설정을 하게 된다면 이웃 하나로 현재 데이터를 판단하기 때문에 너무 편향된 정보로 판단하기 때문에 1로 설정을 하지 않는다.
    - 홀수로 설정을 하는 이유는 짝수로 설정을 하는 경우 과반수 이상의 이웃이 나오지 않을 수 있음으로 홀수로 설정한다.
- KNN 알고리즘은 가장 정확도가 높은 n_neighnors를 찾아 해당 값으로 모델을 만든다.

## 거리 구하기 ...........(이미지 추가하기)
- 유클리드 거리(Euclidean Distance) : $ d = \sqrt{(q_1 - p_1)^2 + (q_2 - p_2)^2} $
  
  
- 맨하튼 거리(Manhattan Distance) : $ d = |q_1 - p_1| + |q_2 - p_2| $

KNN에서 탐색하는 이웃의 거리를 구하는 방법으로 어떤 방법을 사용하는지에 따라서 결과값이 다르게 나타난다.  
※ 거리를 구하기 위해서는 변수들을 Scaling 후 사용해야 한다.

## 스케일링
- 정규화(Normalization) : $ x_{norm} = \frac{x - a}{b - a} $
  
  
- 표준화(Standardization) : $ x_z = \frac{x - mean}{std} $

위 공식을 사용하여 Scaling을 진행하거나 함수를 이용할 수 있다.  
ex) 정규화 함수 사용  
> from sklearn.preprocessing import MinMaxScaler  
> scaler = MinMaxScaler()  
> scaler.fit(x_train)  
> x_train = scaler.transform(x_train)  
> x_test = scaler.transform(x_test)

## 실습

In [48]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format='retina'

- survivied: 생존여부 (1: 생존, 0: 사망)
- pclass: 좌석 등급 (1등급, 2등급, 3등급)
- sex: 성별
- age: 나이
- sibsp: 형제 + 배우자 수
- parch: 부모 + 자녀 수
- fare: 좌석 요금
- embarked: 탑승 항구 (S, C, Q)
- class: pclass와 동일
- who: 성별과 동일
- adult_male: 성인 남자 여부
- deck: 데크 번호 (알파벳 + 숫자 혼용)
- embark_town: 탑승 항구 이름
- alive: 생존여부 (yes, no)
- alone: 혼자 탑승 여부

In [49]:
data = sns.load_dataset('titanic')

In [50]:
data.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [52]:
drop_cols = ['deck', 'alive', 'alone', 'adult_male', 'sibsp', 'parch', 'who', 'embark_town']
data.drop(drop_cols, axis = 1, inplace=True)
data.head()

,survived,pclass,sex,age,fare,embarked,class
0,0,3,male,22.0,7.2500,S,Third
1,1,1,female,38.0,71.2833,C,First
2,1,3,female,26.0,7.9250,S,Third
3,1,1,female,35.0,53.1000,S,First
4,0,3,male,35.0,8.0500,S,Third


In [53]:
data.isna().sum()

survived      0
pclass        0
sex           0
age         177
fare          0
embarked      2
class         0
dtype: int64

In [54]:
mean_age = data['age'].mean()
data['age'].fillna(mean_age, inplace=True)

In [55]:
data['embarked'].value_counts()

S    644
C    168
Q     77
Name: embarked, dtype: int64

In [56]:
data['embarked'].fillna('S', inplace=True)
data.isna().sum()

survived    0
pclass      0
sex         0
age         0
fare        0
embarked    0
class       0
dtype: int64

In [59]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   survived  891 non-null    int64   
 1   pclass    891 non-null    int64   
 2   sex       891 non-null    object  
 3   age       891 non-null    float64 
 4   fare      891 non-null    float64 
 5   embarked  891 non-null    object  
 6   class     891 non-null    category
dtypes: category(1), float64(2), int64(2), object(2)
memory usage: 42.9+ KB


In [60]:
dumm_cols = ['sex', 'embarked', 'class']

In [61]:
target = 'survived'
x = data.drop(target, axis = 1)
y = data.loc[:, target]

In [62]:
x = pd.get_dummies(x, columns=dumm_cols, drop_first=True)

In [63]:
x.head()

,pclass,age,fare,sex_male,embarked_Q,embarked_S,class_Second,class_Third
0,3,22.0,7.2500,1,0,1,0,1
1,1,38.0,71.2833,0,0,0,0,0
2,3,26.0,7.9250,0,0,1,0,1
3,1,35.0,53.1000,0,0,1,0,0
4,3,35.0,8.0500,1,0,1,0,1


In [64]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [65]:
# 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [66]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report

In [67]:
# 선언하기
model = KNeighborsClassifier()

In [71]:
# 학습하기
model.fit(x_train, y_train)

KNeighborsClassifier()

In [72]:
# 예측하기
y_pred = model.predict(x_test)

In [73]:
# 평가하기
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[142  22]
 [ 28  76]]
              precision    recall  f1-score   support

           0       0.84      0.87      0.85       164
           1       0.78      0.73      0.75       104

    accuracy                           0.81       268
   macro avg       0.81      0.80      0.80       268
weighted avg       0.81      0.81      0.81       268

